## Load data

In [1]:
import pandas as pd
import numpy as np

In [8]:
data = pd.read_csv('lending-club-data.csv')
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1,1,1,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1,1,1,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1,1,1,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1,1,1,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1,1,1,0,5.21533,20141201T000000,1,1,1


In [9]:
data['safe_loans'] = data['bad_loans'].apply(lambda x : +1 if x==0 else -1)
data.drop('bad_loans', axis = 1, inplace = True)

## Subset Features

In [7]:
features_names = ['grade',              # grade of the loan
                  'term',               # the term of the loan
                  'home_ownership',     # home_ownership status: own, mortgage or rent
                  'emp_length',         # number of years of employment
                 ]
target_name = 'safe_loans'

In [11]:
features = data[features_names]
target = data[target_name]

In [12]:
def read_index(filename):
    with open(filename, 'r') as f:
        first_line = f.readline()
    first_line = first_line.translate(None,'[]"').strip().split(',')
    first_line = [x.strip() for x in first_line]
    return first_line

train_idx = read_index('train-idx.json')
validation_idx = read_index('valid-idx.json')
train_idx = [int(x) for x in train_idx]
validation_idx = [int(x) for x in validation_idx]

In [13]:
features = pd.get_dummies(features)
features[target_name] = target

In [14]:
train_data = features.iloc[train_idx]
validation_data = features.iloc[validation_idx]

In [15]:
train_data.reset_index()
validation_data.reset_index()
print features.columns.values

['grade_A' 'grade_B' 'grade_C' 'grade_D' 'grade_E' 'grade_F' 'grade_G'
 'term_ 36 months' 'term_ 60 months' 'home_ownership_MORTGAGE'
 'home_ownership_OTHER' 'home_ownership_OWN' 'home_ownership_RENT'
 'emp_length_1 year' 'emp_length_10+ years' 'emp_length_2 years'
 'emp_length_3 years' 'emp_length_4 years' 'emp_length_5 years'
 'emp_length_6 years' 'emp_length_7 years' 'emp_length_8 years'
 'emp_length_9 years' 'emp_length_< 1 year' 'emp_length_n/a' 'safe_loans']


## Regularised decision tree implementation

In [16]:
def reached_minimum_node_size(data, min_node_size):
    if data.shape[0] <= min_node_size:
        return True
    return False

In [17]:
def error_reduction(error_before_split, error_after_split):
    return error_before_split-error_after_split

## Quiz question: Given an intermediate node with 6 safe loans and 3 risky loans, if the min_node_size parameter is 10, what should the tree learning algorithm do next?

Stop tree growing process

#Quiz question: Assume an intermediate node has 6 safe loans and 3 risky loans. For each of 4 possible features to split on, the error reduction is 0.0, 0.05, 0.1, and 0.14, respectively. If the minimum gain in error reduction parameter is set to 0.2, what should the tree learning algorithm do next?

Stop tree growing process

## Function to compute intermidiate num mistakes

In [18]:
def intermediate_node_num_mistakes(labels_in_node):
    if len(labels_in_node) == 0:
        return 0    
    n_plus = sum(labels_in_node == 1)
    n_minus = sum(labels_in_node == -1)
    if n_plus >= n_minus:
        return n_minus
    return n_plus

## Function to compute best  splitting feature

In [19]:
def best_splitting_feature(data, features, target):
    
    target_values = data[target]
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    
    num_data_points = float(len(data))  
    
    for feature in features:
        
        left_split = data[data[feature] == 0]
        right_split = data[data[feature] == 1]
            
        left_mistakes = intermediate_node_num_mistakes(left_split[target])   
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
        
        error = (left_mistakes + right_mistakes) / num_data_points
        
        
        if error < best_error:
            best_feature = feature
            best_error = error
    
    return best_feature 

## Function to create new leaf

In [20]:
def create_leaf(target_values):    
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True}   
   
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])    

    if num_ones > num_minus_ones:
        leaf['prediction'] =  1       
    else:
        leaf['prediction'] = -1            

    return leaf 

## Function decision tree create

In [30]:
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0):
    
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    
    
    # Stopping condition 1: All nodes are of the same type.
    if intermediate_node_num_mistakes(target_values) == 0:
        print "Stopping condition 1 reached. All data points have the same target value."                
        return create_leaf(target_values)
    
    # Stopping condition 2: No more features to split on.
    if remaining_features == 0:
        print "Stopping condition 2 reached. No remaining features."                
        return create_leaf(target_values)    
    
    # Early stopping condition 1: Reached max depth limit.
    if current_depth >= max_depth:
        print "Early stopping condition 1 reached. Reached maximum depth."
        return create_leaf(target_values)
    
    # Early stopping condition 2: Reached the minimum node size.
    # If the number of data points is less than or equal to the minimum size, return a leaf.
    if reached_minimum_node_size(data, min_node_size):
        print "Early stopping condition 2 reached. Reached minimum node size."
        return create_leaf(target_values)
    
    # Find the best splitting feature
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    # Early stopping condition 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples 
    # divided by the total number of examples)
    error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))
    
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    left_mistakes = intermediate_node_num_mistakes(target_values[data[splitting_feature] == 0])
    right_mistakes = intermediate_node_num_mistakes(target_values[data[splitting_feature] == 1])
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, return a leaf.
    if error_reduction(error_before_split, error_after_split) < min_error_reduction:
        print "Early stopping condition 3 reached. Minimum error reduction."
        return create_leaf(target_values)
    
    
    remaining_features.remove(splitting_feature)
    print "Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split))
    
    
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)        
    
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)  
    
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

## Learning

In [31]:
my_decision_tree_new = decision_tree_create(train_data, list(features.columns.values[:-1]), 'safe_loans', max_depth = 6, min_node_size = 100, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
E

In [33]:
my_decision_tree_old = decision_tree_create(train_data, list(features.columns.values[:-1]), 'safe_loans', max_depth = 6, min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
E

## Predictions

In [34]:
def classify(tree, x, annotate = False):
       # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate:
             print "At leaf, predicting %s" % tree['prediction']
        return tree['prediction']
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
             print "Split on %s = %s" % (tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

In [36]:
validtation_data = validation_data.reset_index()

In [39]:
print validation_data.iloc[0]
print 'Predicted class: %s ' % classify(my_decision_tree_new, validation_data.iloc[0])

grade_A                    0
grade_B                    0
grade_C                    0
grade_D                    1
grade_E                    0
grade_F                    0
grade_G                    0
term_ 36 months            0
term_ 60 months            1
home_ownership_MORTGAGE    0
home_ownership_OTHER       0
home_ownership_OWN         0
home_ownership_RENT        1
emp_length_1 year          0
emp_length_10+ years       0
emp_length_2 years         1
emp_length_3 years         0
emp_length_4 years         0
emp_length_5 years         0
emp_length_6 years         0
emp_length_7 years         0
emp_length_8 years         0
emp_length_9 years         0
emp_length_< 1 year        0
emp_length_n/a             0
safe_loans                -1
Name: 24, dtype: float64
Predicted class: -1 


In [40]:
classify(my_decision_tree_new, validation_data.iloc[0], annotate = True)


Split on term_ 36 months = 0.0
Split on grade_A = 0.0
Split on grade_B = 0.0
Split on grade_C = 0.0
Split on grade_D = 1.0
Split on grade_E = 0.0
At leaf, predicting -1


-1

In [41]:
classify(my_decision_tree_old, validation_data.iloc[0], annotate = True)

Split on term_ 36 months = 0.0
Split on grade_A = 0.0
Split on grade_B = 0.0
Split on grade_C = 0.0
Split on grade_D = 1.0
Split on grade_E = 0.0
At leaf, predicting -1


-1

##Quiz question: For my_decision_tree_new trained with max_depth = 6, min_node_size = 100, min_error_reduction=0.0, is the prediction path for validation_set[0] shorter, longer, or the same as for my_decision_tree_old that ignored the early stopping conditions 2 and 3?

The same....

##Quiz question: For my_decision_tree_new trained with max_depth = 6, min_node_size = 100, min_error_reduction=0.0, is the prediction path for any point always shorter, always longer, always the same, shorter or the same, or longer or the same as for my_decision_tree_old that ignored the early stopping conditions 2 and 3?

shorter or the same

##Quiz question: For a tree trained on any dataset using max_depth = 6, min_node_size = 100, min_error_reduction=0.0, what is the maximum number of splits encountered while making a single prediction?

6

## Evaluating Decision Tree

In [42]:
def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    prediction = []
    for i in range(data.shape[0]):
        prediction.append(classify(tree, data.iloc[i]))
    
    errors = np.sum(prediction != data[target_name])
    return errors / float(len(prediction))

In [44]:
evaluate_classification_error(my_decision_tree_new, validation_data)


0.38410168031021114

In [46]:
evaluate_classification_error(my_decision_tree_old, validation_data)


0.38377854373115039

## Exploring the depth 

In [48]:
model_1 = decision_tree_create(train_data, list(features.columns.values[:-1]), 'safe_loans', 
                                            max_depth = 2, min_node_size = 0, min_error_reduction=-1.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 1 (28001 data points).
Split on feature grade_D. (23300, 4701)
--------------------------------------------------------------------
Subtree, depth = 2 (23300 data points).
Early stopping condition 1 reached. Reached maximum depth.
-----------------------------------------------

In [49]:
model_2 = decision_tree_create(train_data, list(features.columns.values[:-1]), 'safe_loans',
                                            max_depth = 6, min_node_size = 0.0, min_error_reduction=-1.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
E

In [50]:
model_3 = decision_tree_create(train_data, list(features.columns.values[:-1]), 'safe_loans',
                                            max_depth = 14, min_node_size = 0.0, min_error_reduction=-1.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
S

In [51]:
print "Training data, classification error (model 1):", evaluate_classification_error(model_1, train_data)
print "Training data, classification error (model 2):", evaluate_classification_error(model_2, train_data)
print "Training data, classification error (model 3):", evaluate_classification_error(model_3, train_data)

Training data, classification error (model 1): 0.400037610144
Training data, classification error (model 2): 0.381850419084
Training data, classification error (model 3): 0.374462712229


In [52]:
print "Validation data, classification error (model 1):", evaluate_classification_error(model_1, validation_data)
print "Validation data, classification error (model 2):", evaluate_classification_error(model_2, validation_data)
print "Validation data, classification error (model 3):", evaluate_classification_error(model_3, validation_data)

Validation data, classification error (model 1): 0.398104265403
Validation data, classification error (model 2): 0.383778543731
Validation data, classification error (model 3): 0.380008616975


##Quiz Question: Which tree has the smallest error on the validation data?

model 3

##Quiz Question: Does the tree with the smallest error in the training data also have the smallest error in the validation data?

Yes

##Quiz Question: Is it always true that the tree with the lowest classification error on the training set will result in the lowest classification error in the validation set?

No

In [53]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

In [55]:
print count_leaves(model_1)
print count_leaves(model_2)
print count_leaves(model_3)

4
41
341


##Quiz question: Which tree has the largest complexity?

model_3

##Quiz question: Is it always true that the most complex tree will result in the lowest classification error in the validation_set?

No

In [59]:
model_4 = decision_tree_create(train_data, list(features.columns.values[:-1]), 'safe_loans',
                                            max_depth = 6, min_node_size = 0.0, min_error_reduction=-1.0)
model_5 = decision_tree_create(train_data, list(features.columns.values[:-1]), 'safe_loans',
                                            max_depth = 6, min_node_size = 0.0, min_error_reduction=0.0)
model_6 = decision_tree_create(train_data, list(features.columns.values[:-1]), 'safe_loans',
                                            max_depth = 6, min_node_size = 0.0, min_error_reduction=5.0)

 --------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).


In [64]:
print "Validation data, classification error (model 4):", evaluate_classification_error(model_4, validation_data)
print "Validation data, classification error (model 5):", evaluate_classification_error(model_5, validation_data)
print "Validation data, classification error (model 6):", evaluate_classification_error(model_6, validation_data)
print "model 4 complexity: ", count_leaves(model_4)
print "model 5 complexity: ", count_leaves(model_5)
print "model 6 complexity: ", count_leaves(model_6)

Validation data, classification error (model 4): 0.383778543731
Validation data, classification error (model 5): 0.383778543731
Validation data, classification error (model 6): 0.503446790177
model 4 complexity:  41
model 5 complexity:  41
model 6 complexity:  1


##Quiz Question: Using the complexity definition above, which model (model_4, model_5, or model_6) has the largest complexity? Did this match your expectation?

model_4

##Quiz Question: model_4 and model_5 have similar classification error on the validation set but model_5 has lower complexity? Should you pick model_5 over model_4?

model_4

In [62]:
model_7 = decision_tree_create(train_data, list(features.columns.values[:-1]), 'safe_loans',
                                            max_depth = 6, min_node_size = 0.0, min_error_reduction=-1.0)
model_8 = decision_tree_create(train_data, list(features.columns.values[:-1]), 'safe_loans',
                                            max_depth = 6, min_node_size = 2000.0, min_error_reduction=-1.0)
model_9 = decision_tree_create(train_data, list(features.columns.values[:-1]), 'safe_loans',
                                            max_depth = 6, min_node_size = 50000.0, min_error_reduction=-1.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
E

In [63]:
print "Validation data, classification error (model 7):", evaluate_classification_error(model_7, validation_data)
print "Validation data, classification error (model 8):", evaluate_classification_error(model_8, validation_data)
print "Validation data, classification error (model 9):", evaluate_classification_error(model_9, validation_data)
print "Model 7 complexity: ", count_leaves(model_7)
print "Model 8 complexity: ", count_leaves(model_8)
print "Model 9 complexity: ", count_leaves(model_9)



Validation data, classification error (model 7): 0.383778543731
Validation data, classification error (model 8): 0.384532529082
Validation data, classification error (model 9): 0.503446790177
Model 7 complexity:  41
Model 8 complexity:  19
Model 9 complexity:  1


##Quiz Question: Using the results obtained in this section, which model (model_7, model_8, or model_9) would you choose to use?

model_8